In [5]:
import os
import numpy as np
import cv2

In [6]:
path_A = r"D:\DELFT\Q4 GEO1101 Synthesis\ML\tiles\zonatlas\ahn3_context\18"
path_B = r"D:\DELFT\Q4 GEO1101 Synthesis\ML\tiles\zonatlas\radiation\18"
path_AB = r"D:\DELFT\Q4 GEO1101 Synthesis\ML\tiles\zonatlas_roofs_AB_radiation"
shape = (256,256,3)

if not os.path.exists(path_AB):
    os.makedirs(path_AB)

def build_filedict(path):
  filedict = {}
  for sub in os.walk(path):
    
    subdir = sub[0]
    files = sub[2]
    dirname = os.path.basename(subdir)
    for file in files:
      filename = dirname + "_" + file
      filepath = subdir + "\\" + file
      filedict[filename] = filepath
  return filedict

imgs_A = build_filedict(path_A)
imgs_B = build_filedict(path_B)

In [3]:
def read_transparent_png(filename):
    image_4channel = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    alpha_channel = image_4channel[:,:,3]
    rgb_channels = image_4channel[:,:,:3]

    # White Background Image
    white_background_image = np.ones_like(rgb_channels, dtype=np.uint8) * 255

    # Alpha factor
    alpha_factor = alpha_channel[:,:,np.newaxis].astype(np.float32) / 255.0
    alpha_factor = np.concatenate((alpha_factor,alpha_factor,alpha_factor), axis=2)

    # Transparent Image Rendered on White Background
    base = rgb_channels.astype(np.float32) * alpha_factor
    white = white_background_image.astype(np.float32) * (1 - alpha_factor)
    final_image = base + white
    return final_image.astype(np.uint8)

In [7]:
for img, path in imgs_A.items():
  try:
    A = cv2.imread(imgs_A[img])
    try:
      B = read_transparent_png(imgs_B[img])
    except:
      B = cv2.imread(imgs_B[img])
  except KeyError as e:
    continue
  
  # Skip empty (white) image pairs
  if np.mean(A) > 254:
    continue
  
  # Skip invalid shaped images
  if A.shape != shape or B.shape != shape:
    continue
    
  AB = np.concatenate([A, B], 1)
  export_path = path_AB + "\\" + img
  cv2.imwrite(export_path, AB)